In [1]:
import os
import glob 
import pandas as pd



In [2]:
pattern = os.path.join('..', 'data', 'prepared', '*.csv')
print(pattern)
files = glob.glob(pattern)
print(files)

../data/prepared/*.csv
['../data/prepared/communes-france-2025.csv', '../data/prepared/ValeursFoncieres-2025-S1.csv']


In [3]:
df = pd.read_csv(files[1], sep=';')
df.head()


,id_transaction,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,...,Code departement,Code commune,Prefixe de section,Section,No plan,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Surface terrain
0,0,2025-01-01 00:00:00,Vente,30000,13.0,NaN,GR,0005,LA GRANDE RUE,55140.0,...,55,474,NaN,AA,320,1,Maison,79,3,603
1,1,2025-01-01 00:00:00,Vente,77800,336.0,NaN,RUE,0065,DE LA HAUTE LANDE,40120.0,...,40,245,NaN,AR,44,1,Maison,97,5,789
2,3,2025-01-02 00:00:00,Vente,20000,7.0,NaN,NaN,B173,PONS,19290.0,...,19,261,NaN,E,534,1,Maison,64,4,507
3,4,2025-01-02 00:00:00,Vente,20000,15.0,NaN,RUE,0090,ODE AN DENVED,29233.0,...,29,30,NaN,AH,25,1,Maison,28,1,114
4,5,2025-01-02 00:00:00,Vente,20000,26.0,NaN,BD,1805,DE LA GOBLECHERE,79300.0,...,79,49,NaN,AP,183,1,Maison,24,1,193


In [4]:
df2 = pd.read_csv(files[0], sep=';')
df2.head()

,code_insee,nom_standard,population,superficie_km2,densite,altitude_moyenne,latitude_centre,longitude_centre,niveau_equipements_services
0,01001,L'Abergement-Clémenciat,832,16.0,53.0,242,46.153000,4.926,0.0
1,01002,L'Abergement-de-Varey,267,9.0,29.0,483,46.008999,5.428,0.0
2,01004,Ambérieu-en-Bugey,14854,24.0,607.0,379,45.960999,5.373,3.0
3,01005,Ambérieux-en-Dombes,1897,16.0,118.0,290,45.995998,4.912,1.0
4,01006,Ambléon,113,6.0,19.0,589,45.750000,5.594,0.0


In [5]:
df2[df2['latitude_centre'].isna()]

,code_insee,nom_standard,population,superficie_km2,densite,altitude_moyenne,latitude_centre,longitude_centre,niveau_equipements_services


In [6]:
df2.loc[df2['latitude_centre'].isna(), 'latitude_centre'] = 0.0
df2.loc[df2['longitude_centre'].isna(), 'longitude_centre'] = 0.0

df2[df2['longitude_centre'].isna()]

,code_insee,nom_standard,population,superficie_km2,densite,altitude_moyenne,latitude_centre,longitude_centre,niveau_equipements_services


In [7]:
# On cree la colonne 'code insee'
def safe_int_conversion(x):
    try:
        return int(x)
    except (ValueError, TypeError):
        return None

def compute_code_insee(row):
    code_dept = safe_int_conversion(row['Code departement'])
    code_commune = safe_int_conversion(row['Code commune'])
    if code_dept is not None and code_commune is not None:
        return str((code_dept * 1000) + code_commune).zfill(5)
    return '00000'

df['Code insee'] = df.apply(compute_code_insee, axis=1)



In [8]:
#df.drop(columns=['code insee'], inplace=True)
df.head()

,id_transaction,Date mutation,Nature mutation,Valeur fonciere,No voie,B/T/Q,Type de voie,Code voie,Voie,Code postal,...,Code commune,Prefixe de section,Section,No plan,Code type local,Type local,Surface reelle bati,Nombre pieces principales,Surface terrain,Code insee
0,0,2025-01-01 00:00:00,Vente,30000,13.0,NaN,GR,0005,LA GRANDE RUE,55140.0,...,474,NaN,AA,320,1,Maison,79,3,603,55474
1,1,2025-01-01 00:00:00,Vente,77800,336.0,NaN,RUE,0065,DE LA HAUTE LANDE,40120.0,...,245,NaN,AR,44,1,Maison,97,5,789,40245
2,3,2025-01-02 00:00:00,Vente,20000,7.0,NaN,NaN,B173,PONS,19290.0,...,261,NaN,E,534,1,Maison,64,4,507,19261
3,4,2025-01-02 00:00:00,Vente,20000,15.0,NaN,RUE,0090,ODE AN DENVED,29233.0,...,30,NaN,AH,25,1,Maison,28,1,114,29030
4,5,2025-01-02 00:00:00,Vente,20000,26.0,NaN,BD,1805,DE LA GOBLECHERE,79300.0,...,49,NaN,AP,183,1,Maison,24,1,193,79049


In [9]:
df_data = pd.DataFrame()
cols_to_keep = [
    'Valeur fonciere',
    'Surface reelle bati', 
    'Nombre pieces principales', 
    'Surface terrain', 
    'Code insee'
]
df_data = df[cols_to_keep]
df_data.head()


,Valeur fonciere,Surface reelle bati,Nombre pieces principales,Surface terrain,Code insee
0,30000,79,3,603,55474
1,77800,97,5,789,40245
2,20000,64,4,507,19261
3,20000,28,1,114,29030
4,20000,24,1,193,79049


In [10]:
df2_data = pd.DataFrame()
col_to_keep = [
    'code_insee',
    'population',
    'superficie_km2',
    'densite',
    'niveau_equipements_services',
    'latitude_centre',
    'longitude_centre',
    'altitude_moyenne'
]
df2_data = df2[col_to_keep]
#df2_data.loc[:, 'code_insee'] = pd.to_numeric(df2_data['code_insee'], errors='coerce').astype('Int64')
df2_data.head()

,code_insee,population,superficie_km2,densite,niveau_equipements_services,latitude_centre,longitude_centre,altitude_moyenne
0,01001,832,16.0,53.0,0.0,46.153000,4.926,242
1,01002,267,9.0,29.0,0.0,46.008999,5.428,483
2,01004,14854,24.0,607.0,3.0,45.960999,5.373,379
3,01005,1897,16.0,118.0,1.0,45.995998,4.912,290
4,01006,113,6.0,19.0,0.0,45.750000,5.594,589


In [11]:
df_merged = pd.merge(df_data, df2_data, left_on='Code insee', right_on='code_insee')
df_merged.head()

,Valeur fonciere,Surface reelle bati,Nombre pieces principales,Surface terrain,Code insee,code_insee,population,superficie_km2,densite,niveau_equipements_services,latitude_centre,longitude_centre,altitude_moyenne
0,30000,79,3,603,55474,55474,224,17.0,13.0,0.0,48.501999,5.736,333
1,77800,97,5,789,40245,40245,1957,12.0,163.0,2.0,44.036999,-0.327,76
2,20000,64,4,507,19261,19261,764,59.0,13.0,1.0,45.668999,2.180,760
3,20000,28,1,114,29030,29030,3594,38.0,96.0,2.0,48.660000,-4.124,39
4,20000,24,1,193,79049,79049,19906,182.0,109.0,3.0,46.855000,-0.479,175


In [12]:
df_merged.drop(columns=['Code insee', 'code_insee'], inplace=True)
df_merged.head()

,Valeur fonciere,Surface reelle bati,Nombre pieces principales,Surface terrain,population,superficie_km2,densite,niveau_equipements_services,latitude_centre,longitude_centre,altitude_moyenne
0,30000,79,3,603,224,17.0,13.0,0.0,48.501999,5.736,333
1,77800,97,5,789,1957,12.0,163.0,2.0,44.036999,-0.327,76
2,20000,64,4,507,764,59.0,13.0,1.0,45.668999,2.180,760
3,20000,28,1,114,3594,38.0,96.0,2.0,48.660000,-4.124,39
4,20000,24,1,193,19906,182.0,109.0,3.0,46.855000,-0.479,175


In [13]:
df_merged.dtypes

Valeur fonciere                  int64
Surface reelle bati              int64
Nombre pieces principales        int64
Surface terrain                  int64
population                       int64
superficie_km2                 float64
densite                        float64
niveau_equipements_services    float64
latitude_centre                float64
longitude_centre               float64
altitude_moyenne                 int64
dtype: object

Avant ajout coordonnees : RndomForestRegressor : R2=0,23
Apres ajout coordonnees : RndomForestRegressor : R2=0,2459
Apres modif lat et lon with Nan to 0 : R2 = 0.2473

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor


y = df_merged['Valeur fonciere']
X = df_merged.drop(columns=['Valeur fonciere'])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)
preds = model.predict(X_valid)

mae = mean_absolute_error(y_valid, preds)
print(f"MAE = {mae:.0f} €")
mse = mean_squared_error(y_valid, preds)
print(f"MSE = {mse:.0f} €")
r2 = r2_score(y_valid, preds)
print(f"R2 = {r2:.4f}")



MAE = 83674 €
MSE = 178524618229 €
R2 = 0.0336


Avant ajout coordonnees : LinearRegression : R2=0,0387
Apres ajout coordonnees : LinearRegression : error
Apres modif lat et lon with Nan to 0 : R2 = 0,0396

In [27]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train, y_train)
preds = model.predict(X_valid)

mae = mean_absolute_error(y_valid, preds)
print(f"MAE = {mae:.0f} €")
mse = mean_squared_error(y_valid, preds)
print(f"MSE = {mse:.0f} €")
r2 = r2_score(y_valid, preds)
print(f"R2 = {r2:.4f}")

MAE = 137072 €
MSE = 167849391185 €
R2 = 0.0913


Avant ajout coordonnees : DecisionTreeRegressor : R2=-0,2556
Apres ajout coordonnees : DecisionTreeRegressor : R2=-0,4968
Apres modif lat et lon with Nan to 0 : R2 = -0,5040

In [28]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)
preds = model.predict(X_valid)

mae = mean_absolute_error(y_valid, preds)
print(f"MAE = {mae:.0f} €")
mse = mean_squared_error(y_valid, preds)
print(f"MSE = {mse:.0f} €")
r2 = r2_score(y_valid, preds)
print(f"R2 = {r2:.4f}")

MAE = 105304 €
MSE = 274463960382 €
R2 = -0.4858


Avant ajout coordonnees : GradientBoostingRegressor : R2=0,3139
Apres ajout coordonnees : GradientBoostingRegressor : error
Apres modif lat et lon with Nan to 0 : R2 = 0,3481

In [29]:
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor(random_state=42)
model.fit(X_train, y_train)
preds = model.predict(X_valid)

mae = mean_absolute_error(y_valid, preds)
print(f"MAE = {mae:.0f} €")
mse = mean_squared_error(y_valid, preds)
print(f"MSE = {mse:.0f} €")
r2 = r2_score(y_valid, preds)
print(f"R2 = {r2:.4f}")


MAE = 101060 €
MSE = 134533388912 €
R2 = 0.2717


Avant ajout coordonnees : HistGradientBoostingRegressor : R2=0,1877
Apres ajout coordonnees : HistGradientBoostingRegressor : R2=0,2308
Apres modif lat et lon with Nan to 0 : R2 = 0,2349

In [30]:
from sklearn.ensemble import HistGradientBoostingRegressor
model = HistGradientBoostingRegressor(random_state=42)
model.fit(X_train, y_train)
preds = model.predict(X_valid)

r2 = r2_score(y_valid, preds)
print(f"R2 = {r2:.4f}")

R2 = 0.1866


Avant ajout coordonnees : GradientBoostingRegressor : R2=0,3139
Apres ajout coordonnees : GradientBoostingRegressor : error
Apres modif lat et lon with Nan to 0 : R2 = 0,3481

In [19]:

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor

y = df_merged['Valeur fonciere']
X = df_merged.drop(columns=['Valeur fonciere'])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
model = GradientBoostingRegressor(random_state=42)
model.fit(X_train, y_train)
preds = model.predict(X_valid)

r2 = r2_score(y_valid, preds)
print(f"R2 = {r2:.4f}")



R2 = 0.2717


In [20]:
# Maintenant je vais mettre tout ceci dans une pipeline

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

y = df_merged['Valeur fonciere']
X = df_merged.drop(columns=['Valeur fonciere'])

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', GradientBoostingRegressor(random_state=42))
])

pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_valid)

r2 = r2_score(y_valid, preds)
print(f"R2 = {r2:.4f}")


R2 = 0.2719


In [21]:
# Pour visualiser les données après mise à l'échelle :

scaler = pipeline.named_steps['scaler']

# transforme X_train numérique (assurez-vous que X_train contient uniquement les colonnes numériques)
X_train_scaled = scaler.transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

# reconstruire un DataFrame (facile à inspecter)
import pandas as pd
X_train_scaled_df = pd.DataFrame(X_train_scaled, index=X_train.index)
X_valid_scaled_df = pd.DataFrame(X_valid_scaled, index=X_valid.index)

#X_train.head() 
X_train_scaled_df.head()
#X_train_scaled_df.describe() # moyennes ~0, std ~1

,0,1,2,3,4,5,6,7,8,9
145437,-0.081827,-0.720038,-0.155738,2.502670,-0.111368,1.013197,1.178065,-0.768884,-1.155439,-0.589560
249006,-0.018124,-1.932837,1.039310,-0.470296,-0.235611,-0.200387,-0.382792,0.581398,-1.499369,-0.434752
101231,-0.090927,-0.720038,-0.228117,1.312494,1.415378,-0.503248,1.178065,0.938734,-0.086137,-0.500115
272865,-0.066226,-0.113638,-0.159759,-0.541673,-0.200113,-0.521433,-0.382792,0.142885,-1.782387,-0.606760
47253,-0.025924,0.492761,-0.129199,-0.584986,-0.235611,-0.557342,-1.943649,1.237685,-1.568452,-0.376269


In [22]:
# On integre de la cross validation pour tester sur plusieurs jeu train/valid
from sklearn.model_selection import cross_val_score

# Évaluer avec 5 découpages différents
scores = cross_val_score(pipeline, X, y, cv=5, scoring='r2')

print(f"R2 moyen = {scores.mean():.4f} (+/- {scores.std():.4f})")
print(f"Scores par fold : {scores}")

R2 moyen = 0.3055 (+/- 0.0613)
Scores par fold : [0.37218641 0.21790479 0.31060409 0.25643576 0.37052993]


In [23]:
# Recherche des meilleurs modèles

from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

# Dictionnaire des modèles à tester
models = {
    #'Ridge': Ridge(),
    #'Lasso': Lasso(),
    'Random Forest': RandomForestRegressor(random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(random_state=42),
    'XGBoost' : XGBRegressor()   
}

# Tester chaque modèle
results = {}
for name, model in models.items():
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('model', model)
    ])
    scores = cross_val_score(pipeline, X, y, cv=5, scoring='r2')
    results[name] = scores.mean()
    print(f"{name:20s} : R2 = {scores.mean():.4f} (+/- {scores.std():.4f})")

# Meilleur modèle
best_model = max(results, key=results.get)
print(f"\n🏆 Meilleur modèle : {best_model}")

Random Forest        : R2 = 0.3113 (+/- 0.0906)
Gradient Boosting    : R2 = 0.3055 (+/- 0.0613)
XGBoost              : R2 = 0.2435 (+/- 0.1276)

🏆 Meilleur modèle : Random Forest


In [ ]:
# Recherche des meilleurs hyperparamètres avec GridSearchCV

from sklearn.model_selection import GridSearchCV

# Définir la pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', GradientBoostingRegressor(random_state=42))
])

# Grille de paramètres à tester
param_grid = {
    'model__n_estimators': [50, 100, 200],          # Nombre d'arbres
    'model__learning_rate': [0.01, 0.1, 0.2],       # Taux d'apprentissage
    'model__max_depth': [3, 5, 7]                   # Profondeur des arbres
}

# Recherche des meilleurs paramètres (attention : peut être long (24 min)!)
grid_search = GridSearchCV(
    pipeline, 
    param_grid, 
    cv=5,                    # 5-fold cross-validation
    scoring='r2',
    verbose=1,               # Affiche la progression
    n_jobs=-1                # Utilise tous les CPU
)

grid_search.fit(X_train, y_train)

# Afficher les résultats
print(f"Meilleurs paramètres : {grid_search.best_params_}")
print(f"Meilleur score R2 : {grid_search.best_score_:.4f}")

# Tester sur les données de validation
best_model = grid_search.best_estimator_
preds = best_model.predict(X_valid)
r2_valid = r2_score(y_valid, preds)
print(f"R2 sur validation : {r2_valid:.4f}")

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Meilleurs paramètres : {'model__learning_rate': 0.1, 'model__max_depth': 5, 'model__n_estimators': 200}
Meilleur score R2 : 0.3761
R2 sur validation : 0.2943


Avant ajout coordonnees :
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Meilleurs paramètres : {'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 50}
Meilleur score R2 : 0.1784
R2 sur validation : 0.1861

Apres ajout coordonnees :
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Meilleurs paramètres : {'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 100}
Meilleur score R2 : 0.2147
R2 sur validation : 0.2168

Apres mofid lat et lon :

In [ ]:
# Recherche des meilleurs hyperparamètres avec GridSearchCV

from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# Définir la pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', XGBRegressor(random_state=42))
])

# Grille de paramètres à tester
param_grid = {
    'model__n_estimators': [50, 100, 200],          # Nombre d'arbres
    'model__learning_rate': [0.01, 0.1, 0.2],       # Taux d'apprentissage
    'model__max_depth': [3, 5, 7]                   # Profondeur des arbres
}

# Recherche des meilleurs paramètres (29 sec seulement ?!?)
grid_search = GridSearchCV(
    pipeline, 
    param_grid, 
    cv=5,                    # 5-fold cross-validation
    scoring='r2',
    verbose=1,               # Affiche la progression
    n_jobs=-1                # Utilise tous les CPU
)

grid_search.fit(X_train, y_train)

# Afficher les résultats
print(f"Meilleurs paramètres : {grid_search.best_params_}")
print(f"Meilleur score R2 : {grid_search.best_score_:.4f}")

# Tester sur les données de validation
best_model = grid_search.best_estimator_
preds = best_model.predict(X_valid)
r2_valid = r2_score(y_valid, preds)
print(f"R2 sur validation : {r2_valid:.4f}")

Fitting 5 folds for each of 27 candidates, totalling 135 fits
Meilleurs paramètres : {'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 200}
Meilleur score R2 : 0.2994
R2 sur validation : 0.2010


In [26]:
X_train.head()


,Surface reelle bati,Nombre pieces principales,Surface terrain,population,superficie_km2,densite,niveau_equipements_services,latitude_centre,longitude_centre,altitude_moyenne
145437,37,2,90,261804,50.00,5268.000000,4.0,44.856998,-0.574,9
249006,86,0,1576,10518,8.00,1280.000000,2.0,48.043999,-1.500,54
101231,30,2,0,161206,566.11,284.760912,4.0,48.887400,2.305,35
272865,49,3,85,4485,20.00,225.000000,2.0,47.008999,-2.262,4
47253,80,4,123,824,8.00,107.000000,0.0,49.592999,-1.686,71
